In [2]:
import sentinel1decoder
import pandas as pd
import numpy as np
import logging
import math
import cmath
import struct
import matplotlib.pyplot as plt
from matplotlib import colors
from scipy.interpolate import interp1d

In [3]:
inputfile = "/Users/roberto.delprete/Library/CloudStorage/OneDrive-ESA/Desktop/Repos/SARPYX/data/S1A_S1_RAW__0SDH_20240502T121147_20240502T121217_053692_06859D_BB61.SAFE/s1a-s1-raw-s-hh-20240502t121147-20240502t121217-053692-06859d.dat"
l0file = sentinel1decoder.Level0File(inputfile)

In [13]:
selected_burst = 8
selection = l0file.get_burst_metadata(selected_burst)
selection

,Packet Version Number,Packet Type,Secondary Header Flag,PID,PCAT,Sequence Flags,Packet Sequence Count,Packet Data Length,Coarse Time,Fine Time,...,SWL,SAS SSB Flag,Polarisation,Temperature Compensation,Calibration Mode,Tx Pulse Number,Signal Type,Swap Flag,Swath Number,Number of Quads
Packet Number,,,,,,,,,,,,,,,,,,,,,
408,0,0,1,65,12,3,408,17306,1398687125,0.642448,...,0.000258,0,3,3,0,0,0,0,0,12862
409,0,0,1,65,12,3,409,17118,1398687125,0.642982,...,0.000258,0,3,3,0,0,0,0,0,12862
410,0,0,1,65,12,3,410,17098,1398687125,0.643517,...,0.000258,0,3,3,0,0,0,0,0,12862
411,0,0,1,65,12,3,411,17190,1398687125,0.644066,...,0.000258,0,3,3,0,0,0,0,0,12862
412,0,0,1,65,12,3,412,17270,1398687125,0.644600,...,0.000258,0,3,3,0,0,0,0,0,12862
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56533,0,0,1,65,12,3,7381,18050,1398687155,0.645287,...,0.000258,0,3,3,0,0,0,1,0,12862
56534,0,0,1,65,12,3,7382,17958,1398687155,0.645821,...,0.000258,0,3,3,0,0,0,1,0,12862
56535,0,0,1,65,12,3,7383,18078,1398687155,0.646355,...,0.000258,0,3,3,0,0,0,1,0,12862


In [14]:
def extract_required_columns(metadata: pd.DataFrame) -> pd.DataFrame:
    """
    Extract required columns from the metadata DataFrame.

    Args:
        metadata (pd.DataFrame): The metadata DataFrame.

    Returns:
        pd.DataFrame: DataFrame containing only the required columns.
    """
    required_columns = [
        'range_decimation',
        'tx_pulse_start_freq',
        'tx_ramp_rate',
        'tx_pulse_length'
    ]
    for col in required_columns:
        assert col in metadata.columns, f'Missing required column: {col}'
    return metadata[required_columns]


In [17]:
def print_required_columns_values(metadata: pd.DataFrame, required_columns: list[str]) -> None:
    """
    Print the values of the required columns for the first row (iloc 0) of the metadata DataFrame.

    Args:
        metadata (pd.DataFrame): The metadata DataFrame.
        required_columns (list[str]): List of required column names.

    Returns:
        None
    """
    assert isinstance(metadata, pd.DataFrame), 'metadata must be a pandas DataFrame'
    assert isinstance(required_columns, list), 'required_columns must be a list'
    for col in required_columns:
        assert col in metadata.columns, f'Missing required column: {col}'
    values = metadata.iloc[0][required_columns]
    for col, val in values.items():
        print(f'{col}: {val}')

print_required_columns_values(selection, required_columns)

Range Decimation: 1.0
Tx Pulse Start Frequency: -43801344.99740284
Tx Ramp Rate: 1927378686406.996
Tx Pulse Length: 4.5451248822135946e-05


In [7]:
required_columns = [
       'Range Decimation',
       'Tx Pulse Start Frequency',
       'Tx Ramp Rate',
       'Tx Pulse Length'
]
